## Title: Koppen-Geiger Climate Changes - 1901 - 2100

### Description
Köppen climate classification is a widely used vegetation-based empirical climate classification system developed by German botanist-climatologist Wladimir Köppen. It's based on the idea that climate is best defined by native vegetation. This particular version of Köppen-Geiger shows the changes in the climate regions and biomes over time (from 1901 - 2100) using observations for 1901 - 2000 and the A1F1 IPCC climate scenario for 2001 - 2100. A1F1 is the worst case scenario future projection for climate change modeling. You can find out more information on how this was made by visiting World Maps of Köppen-Geiger Climate Classisification website.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/
Cordinate system EPSG: 4326 (WGS84)
Size: 40 MB
Format: geoJSON

### Original source
Original Source: http://koeppen-geiger.vu-wien.ac.at/shifts.htm

### Licence
Only non-commercial scientific or educational use. Please attribute using citation.

### Citation
Rubel, F., and M. Kottek, 2010: Observed and projected climate shifts 1901-2100 depicted by world maps of the Köppen-Geiger climate classification. Meteorol. Z. , 19, 135-141. DOI: 10.1127/0941-2948/2010/0430. http://koeppen-geiger.vu-wien.ac.at/pdf/Paper_2010.pdf

### Original Format
Global coverage, vector, shapefile
Cordinate system: Bassel

### Metadata
Climate scenarios available
Observed:	1901-1925	1926-1950	1951-1975	1976-2000
A1FI:	2001-2025	2026-2050	2051-2075	2076-2100
A2:	2001-2025	2026-2050	2051-2075	2076-2100
B1:	2001-2025	2026-2050	2051-2075	2076-2100
B2:	2001-2025	2026-2050	2051-2075	2076-2100
Each contains vector data for the respective period and emission scenario. Use the scheme below to convert the integers to climate classes:
11 ... Af
12 ... Am
13 ... As
14 ... Aw
21 ... BWk
22 ... BWh
26 ... BSk
27 ... BSh
31 ... Cfa
32 ... Cfb
33 ... Cfc
34 ... Csa
35 ... Csb
36 ... Csc
37 ... Cwa
38 ... Cwb
39 ... Cwc
41 ... Dfa
42 ... Dfb
43 ... Dfc
44 ... Dfd
45 ... Dsa
46 ... Dsb
47 ... Dsc
48 ... Dsd
49 ... Dwa
50 ... Dwb
51 ... Dwc
52 ... Dwd
61 ... EF
62 ... ET

### Notes
The original layers contain topological errors (non-simple geometry) - and an overlapping region at the international time/date line. Manual editting removed the few overlapping vertices.

### Processing
Repair geometry, fix topologial error (remove overlaps), convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
# Import arcpy module
import arcpy

# Local variables:
in_shp = r"C:/Users/LennyJenny/Documents/ArcGIS/world/UNFCCC/downloads/test/wsrll.shp"
output = "C:/data/WorldSoilResources.geojson"

# output topological fix shapefile if topological error (optional)
in_shp_fix = r"C:/data/wsrll_fix.shp"

# Set environments
arcpy.env.workspace = "c:/data"
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"

# Process: Repair Geometry to remove self intersections and non-simple geometry
arcpy.RepairGeometry_management(in_shp, "DELETE_NULL")

# Process: Count Overlapping Features to determine topological error
# Where count<2 = topological error which will require manual fix or batch clip overlapping areas
countname = r"C:/data/overlap_count.shp"
arcpy.analysis.CountOverlappingFeatures(in_shp, countname, min_overlap_count=2)

# Process: Topoloogical fix: Find overlaps and clip from layer (non-manual, will produce gaps, this can be coded to not leave gaps later)
arcpy.analysis.Intersect(in_shp, "in_memory/intersectbits")
arcpy.analysis.Clip(in_shp, "in_memory/intersectbits", in_shp_fix)

# Process: Features To JSON
arcpy.FeaturesToJSON_conversion(input, output, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON")